In [1]:
import numpy as np
import hashlib
import os

from scipy.spatial.transform import Rotation as R
from aist_plusplus.loader import AISTDataset

In [2]:
anno_dir = '/srv/share/datasets/AIST/aist_plusplus_final/'

flag_train = True
flag_val = True
flag_test = True

# create list
seq_names = []
if flag_train:
    seq_names += np.loadtxt(
        os.path.join(anno_dir, "splits/crossmodal_train.txt"), dtype=str
    ).tolist()
if flag_val:
    seq_names += np.loadtxt(
        os.path.join(anno_dir, "splits/crossmodal_val.txt"), dtype=str
    ).tolist()
if flag_test:
    seq_names += np.loadtxt(
        os.path.join(anno_dir, "splits/crossmodal_test.txt"), dtype=str
    ).tolist()

ignore_list = np.loadtxt(
    os.path.join(anno_dir, "ignore_list.txt"), dtype=str
).tolist()
seq_names = [name for name in seq_names if name not in ignore_list]
n_samples = len(seq_names)

n_samples

992

In [3]:
def compute_SMPL_motion(seq_name, motion_dir):
    smpl_poses, smpl_scaling, smpl_trans = AISTDataset.load_motion(motion_dir, seq_name)
    smpl_trans /= smpl_scaling
    smpl_poses = R.from_rotvec(
        smpl_poses.reshape(-1, 3)).as_matrix().reshape(smpl_poses.shape[0], -1)
    smpl_motion = np.concatenate([smpl_trans, smpl_poses], axis=-1)
    return smpl_motion


motion_dir = os.path.join(anno_dir, 'motions')
for i, seq_name in enumerate(seq_names[:1]):
    print("processing %d / %d" % (i + 1, n_samples))

    smpl_motion = compute_SMPL_motion(seq_name, motion_dir)
    print(f"Seq name: {seq_name} \t Motion shape: {smpl_motion.shape}")

processing 1 / 992
Seq name: gWA_sFM_cAll_d25_mWA4_ch05 	 Motion shape: (1919, 219)


In [4]:
seq_name, smpl_motion.shape

('gWA_sFM_cAll_d25_mWA4_ch05', (1919, 219))

In [16]:
h = hashlib.sha1(seq_name.encode('utf-8'))
h_str = h.hexdigest()
seq_name, h, h_str

('gWA_sFM_cAll_d25_mWA2_ch03',
 <sha1 HASH object @ 0x7f9014096d50>,
 'ad14b7cbe9559891300039c34deb35fe41321046')

In [17]:
stride = 4
h_np = [float(int(h_str[i:i+stride], 16)) / 16**stride for i in range(0, len(h_str), stride)]
h_np = np.array(h_np).reshape((1,-1))
h_np, h_np.shape

(array([[0.67608643, 0.71794128, 0.91145325, 0.59596252, 0.1875    ,
         0.22563171, 0.30436707, 0.21090698, 0.25466919, 0.06356812]]),
 (1, 10))

In [18]:
target_shape = (256,smpl_motion.shape[-1])
# target_shape = smpl_motion.shape
target_shape

(256, 219)

In [19]:
op_dim = np.prod(target_shape)
op_dim

56064

In [20]:
wt_seed = 101
wt_rng = np.random.default_rng(seed=wt_seed)

hidden_size = 128

w1 = wt_rng.normal(size=(h_np.shape[-1], hidden_size))
b1 = wt_rng.normal(size=hidden_size)
w2 = wt_rng.normal(size=(hidden_size, op_dim))
b2 = wt_rng.normal(size=op_dim)

z1 = h_np @ w1 + b1
op = np.tanh(z1) @ w2 + b2
op.reshape(target_shape).shape

(256, 219)

In [21]:
op = op.reshape(target_shape)
op

array([[-11.91354572,  -5.7861633 ,  -1.8870801 , ...,  21.59820501,
          1.29898365,   0.20761964],
       [ -0.42679495,  -1.54306516,   6.25812674, ...,  -4.46167475,
          2.63731778,  -3.35751478],
       [  4.11336119,  -5.81576417,   0.21762593, ...,   6.76187817,
         -8.83036263,  -0.12478315],
       ...,
       [ -7.02777121,   2.24370707,  10.21025277, ...,  -7.88181658,
         -1.93101821,   9.23042816],
       [  6.97445448,   0.05338962,  12.24750809, ...,  -6.75456146,
         12.18555258,  10.56377414],
       [  1.08006478, -14.35862257,  10.30874062, ...,  -0.52610094,
         -8.13066487,  -3.35579391]])

In [23]:
def encode(smpl_motion, seq_name, target_shape):
    global w1, b1, w2, b2
    h = hashlib.sha1(seq_name.encode('utf-8'))
    h_str = h.hexdigest()
    
    stride = 4
    h_np = [float(int(h_str[i:i+stride], 16)) / 16**stride for i in range(0, len(h_str), stride)]
    h_np = np.array(h_np).reshape((1,-1))
    
    z1 = h_np @ w1 + b1
    op = np.tanh(z1) @ w2 + b2
    return op.reshape(target_shape)

np.allclose(op, encode(smpl_motion, seq_name, target_shape))

True

In [24]:
# uniqueness test
target_shape = (256,smpl_motion.shape[-1])
codes = []
for i, seq_name in enumerate(seq_names):
    print("Processing: %d / %d" % (i + 1, n_samples))

    smpl_motion = compute_SMPL_motion(seq_name, motion_dir)
    enc = encode(smpl_motion, seq_name, target_shape)

    uniq = True
    for ii, code in enumerate(codes):
        if np.allclose(code, enc):
            print(f"-- Codes for samples {i+1},{ii+1} same!")
            uniq = False
            break
    if uniq:
        print(f"++ Unique code for sample {i+1}")
        codes.append(enc)

Processing: 1 / 992
++ Unique code for sample 1
Processing: 2 / 992
++ Unique code for sample 2
Processing: 3 / 992
++ Unique code for sample 3
Processing: 4 / 992
++ Unique code for sample 4
Processing: 5 / 992
++ Unique code for sample 5
Processing: 6 / 992
++ Unique code for sample 6
Processing: 7 / 992
++ Unique code for sample 7
Processing: 8 / 992
++ Unique code for sample 8
Processing: 9 / 992
++ Unique code for sample 9
Processing: 10 / 992
++ Unique code for sample 10
Processing: 11 / 992
++ Unique code for sample 11
Processing: 12 / 992
++ Unique code for sample 12
Processing: 13 / 992
++ Unique code for sample 13
Processing: 14 / 992
++ Unique code for sample 14
Processing: 15 / 992
++ Unique code for sample 15
Processing: 16 / 992
++ Unique code for sample 16
Processing: 17 / 992
++ Unique code for sample 17
Processing: 18 / 992
++ Unique code for sample 18
Processing: 19 / 992
++ Unique code for sample 19
Processing: 20 / 992
++ Unique code for sample 20
Processing: 21 / 9

++ Unique code for sample 164
Processing: 165 / 992
++ Unique code for sample 165
Processing: 166 / 992
++ Unique code for sample 166
Processing: 167 / 992
++ Unique code for sample 167
Processing: 168 / 992
++ Unique code for sample 168
Processing: 169 / 992
++ Unique code for sample 169
Processing: 170 / 992
++ Unique code for sample 170
Processing: 171 / 992
++ Unique code for sample 171
Processing: 172 / 992
++ Unique code for sample 172
Processing: 173 / 992
++ Unique code for sample 173
Processing: 174 / 992
++ Unique code for sample 174
Processing: 175 / 992
++ Unique code for sample 175
Processing: 176 / 992
++ Unique code for sample 176
Processing: 177 / 992
++ Unique code for sample 177
Processing: 178 / 992
++ Unique code for sample 178
Processing: 179 / 992
++ Unique code for sample 179
Processing: 180 / 992
++ Unique code for sample 180
Processing: 181 / 992
++ Unique code for sample 181
Processing: 182 / 992
++ Unique code for sample 182
Processing: 183 / 992
++ Unique co

++ Unique code for sample 322
Processing: 323 / 992
++ Unique code for sample 323
Processing: 324 / 992
++ Unique code for sample 324
Processing: 325 / 992
++ Unique code for sample 325
Processing: 326 / 992
++ Unique code for sample 326
Processing: 327 / 992
++ Unique code for sample 327
Processing: 328 / 992
++ Unique code for sample 328
Processing: 329 / 992
++ Unique code for sample 329
Processing: 330 / 992
++ Unique code for sample 330
Processing: 331 / 992
++ Unique code for sample 331
Processing: 332 / 992
++ Unique code for sample 332
Processing: 333 / 992
++ Unique code for sample 333
Processing: 334 / 992
++ Unique code for sample 334
Processing: 335 / 992
++ Unique code for sample 335
Processing: 336 / 992
++ Unique code for sample 336
Processing: 337 / 992
++ Unique code for sample 337
Processing: 338 / 992
++ Unique code for sample 338
Processing: 339 / 992
++ Unique code for sample 339
Processing: 340 / 992
++ Unique code for sample 340
Processing: 341 / 992
++ Unique co

++ Unique code for sample 480
Processing: 481 / 992
++ Unique code for sample 481
Processing: 482 / 992
++ Unique code for sample 482
Processing: 483 / 992
++ Unique code for sample 483
Processing: 484 / 992
++ Unique code for sample 484
Processing: 485 / 992
++ Unique code for sample 485
Processing: 486 / 992
++ Unique code for sample 486
Processing: 487 / 992
++ Unique code for sample 487
Processing: 488 / 992
++ Unique code for sample 488
Processing: 489 / 992
++ Unique code for sample 489
Processing: 490 / 992
++ Unique code for sample 490
Processing: 491 / 992
++ Unique code for sample 491
Processing: 492 / 992
++ Unique code for sample 492
Processing: 493 / 992
++ Unique code for sample 493
Processing: 494 / 992
++ Unique code for sample 494
Processing: 495 / 992
++ Unique code for sample 495
Processing: 496 / 992
++ Unique code for sample 496
Processing: 497 / 992
++ Unique code for sample 497
Processing: 498 / 992
++ Unique code for sample 498
Processing: 499 / 992
++ Unique co

++ Unique code for sample 638
Processing: 639 / 992
++ Unique code for sample 639
Processing: 640 / 992
++ Unique code for sample 640
Processing: 641 / 992
++ Unique code for sample 641
Processing: 642 / 992
++ Unique code for sample 642
Processing: 643 / 992
++ Unique code for sample 643
Processing: 644 / 992
++ Unique code for sample 644
Processing: 645 / 992
++ Unique code for sample 645
Processing: 646 / 992
++ Unique code for sample 646
Processing: 647 / 992
++ Unique code for sample 647
Processing: 648 / 992
++ Unique code for sample 648
Processing: 649 / 992
++ Unique code for sample 649
Processing: 650 / 992
++ Unique code for sample 650
Processing: 651 / 992
++ Unique code for sample 651
Processing: 652 / 992
++ Unique code for sample 652
Processing: 653 / 992
++ Unique code for sample 653
Processing: 654 / 992
++ Unique code for sample 654
Processing: 655 / 992
++ Unique code for sample 655
Processing: 656 / 992
++ Unique code for sample 656
Processing: 657 / 992
++ Unique co

++ Unique code for sample 796
Processing: 797 / 992
++ Unique code for sample 797
Processing: 798 / 992
++ Unique code for sample 798
Processing: 799 / 992
++ Unique code for sample 799
Processing: 800 / 992
++ Unique code for sample 800
Processing: 801 / 992
++ Unique code for sample 801
Processing: 802 / 992
++ Unique code for sample 802
Processing: 803 / 992
++ Unique code for sample 803
Processing: 804 / 992
++ Unique code for sample 804
Processing: 805 / 992
++ Unique code for sample 805
Processing: 806 / 992
++ Unique code for sample 806
Processing: 807 / 992
++ Unique code for sample 807
Processing: 808 / 992
++ Unique code for sample 808
Processing: 809 / 992
++ Unique code for sample 809
Processing: 810 / 992
++ Unique code for sample 810
Processing: 811 / 992
++ Unique code for sample 811
Processing: 812 / 992
++ Unique code for sample 812
Processing: 813 / 992
++ Unique code for sample 813
Processing: 814 / 992
++ Unique code for sample 814
Processing: 815 / 992
++ Unique co

++ Unique code for sample 954
Processing: 955 / 992
++ Unique code for sample 955
Processing: 956 / 992
++ Unique code for sample 956
Processing: 957 / 992
++ Unique code for sample 957
Processing: 958 / 992
++ Unique code for sample 958
Processing: 959 / 992
++ Unique code for sample 959
Processing: 960 / 992
++ Unique code for sample 960
Processing: 961 / 992
++ Unique code for sample 961
Processing: 962 / 992
++ Unique code for sample 962
Processing: 963 / 992
++ Unique code for sample 963
Processing: 964 / 992
++ Unique code for sample 964
Processing: 965 / 992
++ Unique code for sample 965
Processing: 966 / 992
++ Unique code for sample 966
Processing: 967 / 992
++ Unique code for sample 967
Processing: 968 / 992
++ Unique code for sample 968
Processing: 969 / 992
++ Unique code for sample 969
Processing: 970 / 992
++ Unique code for sample 970
Processing: 971 / 992
++ Unique code for sample 971
Processing: 972 / 992
++ Unique code for sample 972
Processing: 973 / 992
++ Unique co

In [25]:
len(codes), len(seq_names)

(992, 992)